# Supervisor (Polyglot)

In [ ]:
#!import ../nbs/Testing.dib

In [ ]:
#r @"../../../../../../../.nuget/packages/fsharp.control.asyncseq/3.2.1/lib/netstandard2.1/FSharp.Control.AsyncSeq.dll"
#r @"../../../../../../../.nuget/packages/system.reactive/6.0.1-preview.1/lib/net6.0/System.Reactive.dll"
#r @"../../../../../../../.nuget/packages/system.reactive.linq/6.0.1-preview.1/lib/netstandard2.0/System.Reactive.Linq.dll"
#r @"../../../../../../../.nuget/packages/argu/6.1.1/lib/netstandard2.0/Argu.dll"
#r @"../../../../../../../.nuget/packages/system.commandline/2.0.0-beta4.22272.1/lib/net6.0/System.CommandLine.dll"
#r @"../../../../../../../.nuget/packages/microsoft.aspnetcore.http.connections.common/7.0.0/lib/net7.0/Microsoft.AspNetCore.Http.Connections.Common.dll"
#r @"../../../../../../../.nuget/packages/microsoft.aspnetcore.http.connections.client/7.0.0/lib/net7.0/Microsoft.AspNetCore.Http.Connections.Client.dll"
#r @"../../../../../../../.nuget/packages/microsoft.aspnetcore.signalr.common/7.0.0/lib/net7.0/Microsoft.AspNetCore.SignalR.Common.dll"
#r @"../../../../../../../.nuget/packages/microsoft.aspnetcore.signalr.client/7.0.0/lib/net7.0/Microsoft.AspNetCore.SignalR.Client.dll"
#r @"../../../../../../../.nuget/packages/microsoft.aspnetcore.signalr.client.core/7.0.0/lib/net7.0/Microsoft.AspNetCore.SignalR.Client.Core.dll"
#r @"../../../../../../../.nuget/packages/fsharp.json/0.4.1/lib/netstandard2.0/FSharp.Json.dll"

In [ ]:
#!import ../nbs/Common.fs
#!import ../nbs/CommonFSharp.fs
#!import ../nbs/Threading.fs
#!import ../nbs/Async.fs
#!import ../nbs/AsyncSeq.fs
#!import ../nbs/Networking.fs
#!import ../nbs/Runtime.fs
#!import ../nbs/FileSystem.fs

In [ ]:
open Common
open FileSystem.Operators
open Microsoft.AspNetCore.SignalR.Client

## sendJson

In [ ]:
let inline sendJson (port : int) (json : string) = async {
    let! portOpen = Networking.testPortOpen port
    if portOpen then
        try
            let connection = HubConnectionBuilder().WithUrl($"http://127.0.0.1:{port}").Build()
            do! connection.StartAsync () |> Async.AwaitTask
            let! result = connection.InvokeAsync<string>("ClientToServerMsg", json) |> Async.AwaitTask
            do! connection.StopAsync () |> Async.AwaitTask
            trace Debug (fun () -> $"sendJson / port: {port} / json: {json} / result.Length: {result |> Option.ofObj |> Option.map String.length}") getLocals
            return Some result
        with ex ->
            trace Critical (fun () -> $"sendJson / port: {port} / json: {json} / ex: {ex |> printException}") getLocals
            return None
    else
        trace Debug (fun () -> "sendJson / error: port not open") getLocals
        return None
}

## sendObj

In [ ]:
let inline sendObj port obj =
    obj
    |> System.Text.Json.JsonSerializer.Serialize
    |> sendJson port

## awaitCompiler

In [ ]:
type VSCPos = {| line : int; character : int |}
type VSCRange = VSCPos * VSCPos
type RString = VSCRange * string
type TracedError = {| trace : string list; message : string |}
type ClientErrorsRes =
    | FatalError of string
    | TracedError of TracedError
    | PackageErrors of {| uri : string; errors : RString list |}
    | TokenizerErrors of {| uri : string; errors : RString list |}
    | ParserErrors of {| uri : string; errors : RString list |}
    | TypeErrors of {| uri : string; errors : RString list |}

In [ ]:
let inline awaitCompiler port cancellationToken = async {
    let! ct =
        cancellationToken
        |> Option.defaultValue System.Threading.CancellationToken.None
        |> Async.mergeCancellationTokenWithDefaultAsync
    
    let cts = new System.Threading.CancellationTokenSource ()

    let compiler = MailboxProcessor.Start (fun inbox -> async {
        let! availablePort = Networking.getAvailablePort (Some 60) port
        if availablePort <> port then
            inbox.Post port
        else
            let repositoryRoot = FileSystem.getSourceDirectory () |> FileSystem.findParent ".paket" false

            let compilerPath =
                repositoryRoot </> "deps/The-Spiral-Language/The Spiral Language 2/artifacts/bin/The Spiral Language 2/release"
                |> System.IO.Path.GetFullPath

            let dllPath = compilerPath </> "Spiral.dll"

            let! exitCode, result =
                Runtime.executeWithOptionsAsync
                    {
                        Command = $@"dotnet ""{dllPath}"" port={availablePort}"
                        CancellationToken = Some ct
                        WorkingDirectory = None
                        OnLine = Some <| fun { Line = line } -> async {
                            if line |> String.contains $"Server bound to: http://localhost:{availablePort}" then
                                do! Networking.waitForPortAccess (Some 500) true availablePort |> Async.Ignore

                                let rec loop retry = async {
                                    let getLocals () = $"port: {availablePort} / retry: {retry} / {getLocals ()}"
                                    try
                                        let pingObj = {| Ping = true |}
                                        let! pingResult = pingObj |> sendObj availablePort
                                        trace Verbose (fun () -> $"awaitCompiler / Ping / result: {pingResult}") getLocals
                                    with ex ->
                                        trace Verbose (fun () -> $"awaitCompiler / Ping / ex: {ex |> printException}") getLocals
                                        do! Async.Sleep 10
                                        do! loop (retry + 1)
                                }
                                do! loop 0
                                inbox.Post availablePort
                        }
                    }
            trace Debug (fun () -> $"awaitCompiler / exitCode: {exitCode} / result: {result}") getLocals
            cts.Cancel ()
    }, ct)

    let! serverPort = compiler.Receive ()

    let connection = HubConnectionBuilder().WithUrl($"http://127.0.0.1:{serverPort}").Build ()
    do! connection.StartAsync () |> Async.AwaitTask

    let event = Event<_> ()
    let disposable = connection.On<string> ("ServerToClientMsg", event.Trigger)
    let stream =
        FSharp.Control.AsyncSeq.unfoldAsync
            (fun () -> async {
                let! msg = event.Publish |> Async.AwaitEvent
                return Some (msg |> FSharp.Json.Json.deserialize<ClientErrorsRes>, ())
            })
            ()

    let disposable =
        newDisposable (fun () ->
            disposable.Dispose ()
            connection.StopAsync () |> Async.AwaitTask |> Async.StartImmediate
        )

    return
        serverPort,
        stream,
        cts.Token,
        disposable
}

## getFileUri

In [ ]:
let inline getFileUri path =
    $"file:///{path |> String.trimStart [| '/' |]}"

In [ ]:
let inline getFilePathFromUri uri =
    match System.Uri.TryCreate (uri, System.UriKind.Absolute) with
    | true, uri -> uri.AbsolutePath |> System.IO.Path.GetFullPath
    | _ -> failwith "invalid uri"

In [ ]:
let inline getCompilerPort () =
    13805

## buildFile

In [ ]:
let inline buildFile timeout port cancellationToken path = async {
    let fullPath = path |> System.IO.Path.GetFullPath
    let fileDir = fullPath |> System.IO.Path.GetDirectoryName
    let fileName = fullPath |> System.IO.Path.GetFileNameWithoutExtension
    let! code = fullPath |> FileSystem.readAllTextAsync

    let stream, disposable = FileSystem.watchDirectory true fileDir
    use _ = disposable

    let token, disposable = Threading.newDisposableToken cancellationToken
    use _ = disposable

    let! serverPort, errors, ct, disposable = awaitCompiler port (Some token)
    use _ = disposable

    let fsxContentSeq =
        stream
        |> FSharp.Control.AsyncSeq.choose (function
            | _, FileSystem.FileSystemChange.Changed (path, Some content) when path = $"{fileName}.fsx" ->
                Some content
            | _ -> None
        )
        |> FSharp.Control.AsyncSeq.map (fun content ->
            Some (content |> String.replace "\r\n" "\n"), None
        )

    let inline printErrorData (data : {| uri : string; errors : RString list |}) =
        let fileName = data.uri |> System.IO.Path.GetFileName
        let errors =
            data.errors
            |> List.map snd
            |> String.concat "\n"
        $"{fileName}:\n{errors}"

    let errorsSeq =
        errors
        |> FSharp.Control.AsyncSeq.choose (fun error ->
            match error with
            | FatalError message ->
                Some (message, error)
            | TracedError data ->
                Some (data.message, error)
            | PackageErrors data when data.errors |> List.isEmpty |> not ->
                Some (data |> printErrorData, error)
            | TokenizerErrors data when data.errors |> List.isEmpty |> not ->
                Some (data |> printErrorData, error)
            | ParserErrors data when data.errors |> List.isEmpty |> not ->
                Some (data |> printErrorData, error)
            | TypeErrors data when data.errors |> List.isEmpty |> not ->
                Some (data |> printErrorData, error)
            | _ -> None
        )
        |> FSharp.Control.AsyncSeq.map (fun (message, error) ->
            None, Some (message, error)
        )

    let outputSeq =
        [ fsxContentSeq; errorsSeq ]
        |> FSharp.Control.AsyncSeq.mergeAll

    let! outputChild =
        ((None, []), outputSeq)
        ||> FSharp.Control.AsyncSeq.scan (
            fun (fsxContentResult, errors) (fsxContent, error) ->
                match fsxContent, error with
                | Some fsxContent, None -> Some fsxContent, errors
                | None, Some error ->
                    fsxContentResult,
                    error :: errors
                | _ -> fsxContentResult, errors
        )
        |> FSharp.Control.AsyncSeq.takeWhileInclusive (fun (fsxContent, errors) ->
            trace Debug (fun () -> $"buildFile / fsxContent: {fsxContent} / errors: {errors}") getLocals
            match fsxContent, errors with
            | None, [] -> true
            | _ -> false
        )
        |> FSharp.Control.AsyncSeq.map (fun (fsxContent, errors) ->
            fsxContent, errors |> List.distinct
        )
        |> FSharp.Control.AsyncSeq.tryLast
        |> Async.withCancellationToken ct
        |> Async.catch
        |> Async.runWithTimeoutAsync timeout
        |> Async.StartChild

    let fileOpenObj = {| FileOpen = {| uri = fullPath |> getFileUri; spiText = code |} |}
    let! _fileOpenResult = fileOpenObj |> sendObj serverPort

    let buildFileObj = {| BuildFile = {| uri = fullPath |> getFileUri; backend = "Fsharp" |} |}
    let! _buildFileResult = buildFileObj |> sendObj serverPort

    return!
        outputChild
        |> Async.map (function
            | Some (Ok (Some result)) -> result
            | Some (Error ex) ->
                trace Critical (fun () -> $"buildFile / error: {ex |> printException}") getLocals
                None, []
            | _ -> None, []
        )
}

## persistCode

In [ ]:
let inline persistCode code = async {
    let tempDir = FileSystem.createTempDirectory ()

    let mainPath = tempDir </> "main.spi"
    do! code |> FileSystem.writeAllTextAsync mainPath

    let repositoryRoot = FileSystem.getSourceDirectory () |> FileSystem.findParent ".paket" false

    let spiprojPath = tempDir </> "package.spiproj"
    let spiprojCode =
        $"""packageDir: {repositoryRoot </> "lib"}
packages:
    |core-
    spiral-
modules:
    main
"""
    do! spiprojCode |> FileSystem.writeAllTextAsync spiprojPath

    let disposable = newDisposable (fun () ->
        ()
        // tempDir |> FileSystem.deleteDirectoryAsync |> Async.Ignore |> Async.RunSynchronously
    )

    return mainPath, disposable
}

## buildCode

In [ ]:
let inline buildCode timeout cancellationToken code = async {
    let! mainPath, disposable = persistCode code
    use _ = disposable
    let port = getCompilerPort ()
    return! mainPath |> buildFile timeout port cancellationToken
}

In [ ]:
//// test

let buildCode timeout cancellationToken code = buildCode timeout cancellationToken code

In [ ]:
//// test

"""inl app () =
    console.write_line "text"
    1i32

inl main () =
    app
    |> dyn
    |> ignore
"""
|> buildCode 15000 None
|> Async.runWithTimeout 15000
|> Option.map (fun (fsxContent, errors) -> fsxContent, errors |> List.map fst)
|> _equal (
    Some (
        Some """let rec closure0 () () : int32 =
    let v0 : string = "text"
    System.Console.WriteLine v0
    1
let v0 : (unit -> int32) = closure0()
()
""",
        []
    )
)

00:00:00 #1 [Debug] createTempDirectory / tempFolder: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20231008-2313-2378-7822-737c8a66580b / result: { CreationTime = 2023-10-08 11:13:23 PM
  Exists = true }
00:00:00 #2 [Debug] runWithTimeoutAsync / timeout: 60
00:00:00 #3 [Debug] executeAsync / options: { Command =
   "dotnet "C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release\Spiral.dll" port=13805"
  WorkingDirectory = None
  CancellationToken = Some System.Threading.CancellationToken
  OnLine = Some <fun:buildCode@3-980> }
00:00:00 #4 [Debug] > pwd: C:\home\git\polyglot\apps\spiral
00:00:00 #5 [Debug] > dll_path: C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release
00:00:01 #6 [Debug] runWithTimeoutAsync / timeout: 500
00:00:01 #7 [Verbose] waitForPortAccess / port: 13805 / retry: 0
00:00:01 #8 [Debug] sendJson / port: 13805 / json: {"Ping":true} / result.Length:
00

In [ ]:
//// test

""
|> buildCode 10000 None
|> Async.runWithTimeout 10000
|> _equal None

00:00:03 #17 [Debug] createTempDirectory / tempFolder: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20231008-2313-2769-6943-68f0ac58a1af / result: { CreationTime = 2023-10-08 11:13:27 PM
  Exists = true }
00:00:03 #18 [Debug] runWithTimeoutAsync / timeout: 60
00:00:03 #19 [Debug] executeAsync / options: { Command =
   "dotnet "C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release\Spiral.dll" port=13805"
  WorkingDirectory = None
  CancellationToken = Some System.Threading.CancellationToken
  OnLine = Some <fun:buildCode@3-980> }
00:00:04 #20 [Debug] > pwd: C:\home\git\polyglot\apps\spiral
00:00:04 #21 [Debug] > dll_path: C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release
00:00:04 #22 [Debug] runWithTimeoutAsync / timeout: 500
00:00:04 #23 [Debug] sendJson / port: 13805 / json: {"Ping":true} / result.Length:
00:00:04 #24 [Verbose] awaitCompiler / Ping / result: Some(n

In [ ]:
//// test

"inl app () =
    0i32

inl a = 1

inl main () =
    app
    |> dyn
    |> ignore
"
|> buildCode 10000 None
|> Async.runWithTimeout 10000
|> Option.map (fun (fsxContent, errors) -> fsxContent, errors |> List.map fst)
|> _equal (
    Some (
        None,
        [ "main.spi:
Global inl/let statements should all return functions known at parse time." ]
    )
)

00:00:14 #34 [Debug] createTempDirectory / tempFolder: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20231008-2313-3794-9460-9e752e43c0ad / result: { CreationTime = 2023-10-08 11:13:37 PM
  Exists = true }
00:00:14 #35 [Debug] runWithTimeoutAsync / timeout: 60
00:00:14 #36 [Debug] executeAsync / options: { Command =
   "dotnet "C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release\Spiral.dll" port=13805"
  WorkingDirectory = None
  CancellationToken = Some System.Threading.CancellationToken
  OnLine = Some <fun:buildCode@3-980> }
00:00:14 #37 [Debug] > pwd: C:\home\git\polyglot\apps\spiral
00:00:14 #38 [Debug] > dll_path: C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release
00:00:14 #39 [Debug] runWithTimeoutAsync / timeout: 500
00:00:15 #41 [Debug] sendJson / port: 13805 / json: {"Ping":true} / result.Length:
00:00:15 #42 [Verbose] awaitCompiler / Ping / result: Some(n

In [ ]:
//// test

"""inl main () =
    1i32 / 0i32
"""
|> buildCode 10000 None
|> Async.runWithTimeout 10000
|> Option.map (fun (fsxContent, errors) -> fsxContent, errors |> List.map fst)
|> _equal (
    Some (
        None,
        [ "An attempt to divide by zero has been detected at compile time." ]
    )
)

00:00:15 #50 [Debug] createTempDirectory / tempFolder: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20231008-2313-3973-7368-7ac2286c68cd / result: { CreationTime = 2023-10-08 11:13:39 PM
  Exists = true }
00:00:16 #51 [Debug] runWithTimeoutAsync / timeout: 60
00:00:16 #52 [Debug] executeAsync / options: { Command =
   "dotnet "C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release\Spiral.dll" port=13805"
  WorkingDirectory = None
  CancellationToken = Some System.Threading.CancellationToken
  OnLine = Some <fun:buildCode@3-980> }
00:00:16 #53 [Debug] > pwd: C:\home\git\polyglot\apps\spiral
00:00:16 #54 [Debug] > dll_path: C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release
00:00:16 #55 [Debug] runWithTimeoutAsync / timeout: 500
00:00:16 #56 [Verbose] waitForPortAccess / port: 13805 / retry: 0
00:00:16 #57 [Debug] sendJson / port: 13805 / json: {"Ping":true} / result.Le

In [ ]:
//// test

"""union a =
    | B
    | c

inl main () =
    ()
"""
|> buildCode 10000 None
|> Async.runWithTimeout 10000
|> Option.map (fun (fsxContent, errors) -> fsxContent, errors |> List.map fst)
|> _equal (
    Some (
        None,
        [ "main.spi:
Expected: uppercase variable" ]
    )
)

00:00:19 #66 [Debug] createTempDirectory / tempFolder: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20231008-2313-4304-0413-0443d4e97a07 / result: { CreationTime = 2023-10-08 11:13:43 PM
  Exists = true }
00:00:19 #67 [Debug] runWithTimeoutAsync / timeout: 60
00:00:19 #68 [Debug] executeAsync / options: { Command =
   "dotnet "C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release\Spiral.dll" port=13805"
  WorkingDirectory = None
  CancellationToken = Some System.Threading.CancellationToken
  OnLine = Some <fun:buildCode@3-980> }
00:00:19 #69 [Debug] > pwd: C:\home\git\polyglot\apps\spiral
00:00:19 #70 [Debug] > dll_path: C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release
00:00:20 #71 [Debug] runWithTimeoutAsync / timeout: 500
00:00:20 #72 [Verbose] waitForPortAccess / port: 13805 / retry: 0
00:00:20 #73 [Debug] sendJson / port: 13805 / json: {"Ping":true} / result.Le

In [ ]:
//// test

"""
/// abc
inl main () =
    ()
"""
|> buildCode 10000 None
|> Async.runWithTimeout 10000
|> Option.map (fun (fsxContent, errors) -> fsxContent, errors |> List.map fst)
|> _equal (
    Some (
        None,
        [ "main.spi:
Expected: whitespace" ]
    )
)

00:00:21 #82 [Debug] createTempDirectory / tempFolder: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20231008-2313-4574-7408-750858514b8e / result: { CreationTime = 2023-10-08 11:13:45 PM
  Exists = true }
00:00:22 #83 [Debug] runWithTimeoutAsync / timeout: 60
00:00:22 #84 [Debug] executeAsync / options: { Command =
   "dotnet "C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release\Spiral.dll" port=13805"
  WorkingDirectory = None
  CancellationToken = Some System.Threading.CancellationToken
  OnLine = Some <fun:buildCode@3-980> }
00:00:22 #85 [Debug] > pwd: C:\home\git\polyglot\apps\spiral
00:00:22 #86 [Debug] > dll_path: C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release
00:00:22 #87 [Debug] runWithTimeoutAsync / timeout: 500
00:00:23 #88 [Debug] sendJson / port: 13805 / json: {"Ping":true} / result.Length:
00:00:23 #89 [Verbose] awaitCompiler / Ping / result: Some(n

In [ ]:
//// test

"""
inl main () =
    real
        inl real_unbox forall a. (obj : a) : a =
            typecase obj with
            | _ => obj
        real_unbox ()
    ()
"""
|> buildCode 10000 None
|> Async.runWithTimeout 10000
|> Option.map (fun (fsxContent, errors) -> fsxContent, errors |> List.map fst)
|> _equal (
    Some (
        None,
        [ "Cannot apply a forall with a term." ]
    )
)

00:00:23 #96 [Debug] createTempDirectory / tempFolder: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20231008-2313-4774-7429-7c2b84f0a940 / result: { CreationTime = 2023-10-08 11:13:47 PM
  Exists = true }
00:00:24 #97 [Debug] runWithTimeoutAsync / timeout: 60
00:00:24 #98 [Debug] executeAsync / options: { Command =
   "dotnet "C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release\Spiral.dll" port=13805"
  WorkingDirectory = None
  CancellationToken = Some System.Threading.CancellationToken
  OnLine = Some <fun:buildCode@3-980> }
00:00:24 #99 [Debug] > pwd: C:\home\git\polyglot\apps\spiral
00:00:24 #100 [Debug] > dll_path: C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release
00:00:24 #101 [Debug] runWithTimeoutAsync / timeout: 500
00:00:24 #102 [Verbose] waitForPortAccess / port: 13805 / retry: 0
00:00:25 #103 [Debug] sendJson / port: 13805 / json: {"Ping":true} / resul

In [ ]:
//// test

"""
inl main () =
    real
        inl real_unbox forall a. (obj : a) : a =
            typecase obj with
            | _ => obj
        real_unbox `i32 1
"""
|> buildCode 10000 None
|> Async.runWithTimeout 10000
|> Option.map (fun (fsxContent, errors) -> fsxContent, errors |> List.map fst)
|> _equal (
    Some (
        None,
        [ "The main function should not have a forall." ]
    )
)

00:00:26 #112 [Debug] createTempDirectory / tempFolder: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20231008-2313-5061-6158-615dcdfacbf5 / result: { CreationTime = 2023-10-08 11:13:50 PM
  Exists = true }
00:00:26 #113 [Debug] runWithTimeoutAsync / timeout: 60
00:00:26 #114 [Debug] executeAsync / options: { Command =
   "dotnet "C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release\Spiral.dll" port=13805"
  WorkingDirectory = None
  CancellationToken = Some System.Threading.CancellationToken
  OnLine = Some <fun:buildCode@3-980> }
00:00:27 #115 [Debug] > pwd: C:\home\git\polyglot\apps\spiral
00:00:27 #116 [Debug] > dll_path: C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release
00:00:27 #117 [Debug] runWithTimeoutAsync / timeout: 500
00:00:27 #118 [Verbose] waitForPortAccess / port: 13805 / retry: 0
00:00:27 #119 [Debug] sendJson / port: 13805 / json: {"Ping":true} / r

In [ ]:
//// test

"""
inl init_series start end inc =
    inl total : f64 = conv ((end - start) / inc) + 1
    listm.init total (conv >> (*) inc >> (+) start) : list f64

type integration = (f64 -> f64) -> f64 -> f64 -> f64

inl integral dt : integration =
    fun f a b =>
        init_series (a + dt / 2) (b - dt / 2) dt
        |> listm.map (f >> (*) dt)
        |> listm.fold (+) 0

inl main () =
    integral 0.1 (fun x => x ** 2) 0 1
"""
|> buildCode 10000 None
|> Async.runWithTimeout 10000
|> Option.map (fun (fsxContent, errors) -> fsxContent, errors |> List.map fst)
|> _equal (
    Some (
        Some "0.3325000000000001\n",
        []
    )
)

00:00:29 #128 [Debug] createTempDirectory / tempFolder: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20231008-2313-5326-2632-25e0353a1c52 / result: { CreationTime = 2023-10-08 11:13:53 PM
  Exists = true }
00:00:29 #129 [Debug] runWithTimeoutAsync / timeout: 60
00:00:29 #130 [Debug] executeAsync / options: { Command =
   "dotnet "C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release\Spiral.dll" port=13805"
  WorkingDirectory = None
  CancellationToken = Some System.Threading.CancellationToken
  OnLine = Some <fun:buildCode@3-980> }
00:00:29 #131 [Debug] > pwd: C:\home\git\polyglot\apps\spiral
00:00:29 #132 [Debug] > dll_path: C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release
00:00:30 #133 [Debug] runWithTimeoutAsync / timeout: 500
00:00:30 #134 [Debug] sendJson / port: 13805 / json: {"Ping":true} / result.Length:
00:00:30 #135 [Verbose] awaitCompiler / Ping / result

In [ ]:
//// test

"""
inl init_series start end inc =
    inl total : f64 = conv ((end - start) / inc) + 1
    listm.init total (conv >> (*) inc >> (+) start) : list f64

type integration = (f64 -> f64) -> f64 -> f64 -> f64

inl integral dt : integration =
    fun f a b =>
        init_series (a + dt / 2) (b - dt / 2) dt
        |> listm.map (f >> (*) dt)
        |> listm.fold (+) 0

inl main () =
    integral 0.01 (fun x => x ** 2) 0 1
"""
|> buildCode 10000 None
|> Async.runWithTimeout 10000
|> Option.map (fun (fsxContent, errors) -> fsxContent, errors |> List.map fst)
// |> _equal None
// |> fun x -> printfn $"{x.ToDisplayString ()}"

Some((, [])) Value (, []) Item1 <null> Item2 [ ]

00:00:32 #143 [Debug] createTempDirectory / tempFolder: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20231008-2313-5640-4036-41eb3575a5d4 / result: { CreationTime = 2023-10-08 11:13:56 PM
  Exists = true }
00:00:32 #144 [Debug] runWithTimeoutAsync / timeout: 60
00:00:32 #145 [Debug] executeAsync / options: { Command =
   "dotnet "C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release\Spiral.dll" port=13805"
  WorkingDirectory = None
  CancellationToken = Some System.Threading.CancellationToken
  OnLine = Some <fun:buildCode@3-980> }
00:00:32 #146 [Debug] > pwd: C:\home\git\polyglot\apps\spiral
00:00:32 #147 [Debug] > dll_path: C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release
00:00:33 #148 [Debug] runWithTimeoutAsync / timeout: 500
00:00:33 #149 [Verbose] waitForPortAccess / port: 13805 / retry: 0
00:00:33 #150 [Debug] sendJson / port: 13805 / json: {"Ping":true} / r

## getFileTokenRange

In [ ]:
let inline getFileTokenRange port cancellationToken path = async {
    let fullPath = path |> System.IO.Path.GetFullPath
    let! code = fullPath |> FileSystem.readAllTextAsync
    let lines = code |> String.split [| '\n' |]

    let token, disposable = Threading.newDisposableToken cancellationToken
    use _ = disposable

    let! serverPort, _errors, ct, disposable = awaitCompiler port (Some token)
    use _ = disposable

    let fileOpenObj = {| FileOpen = {| uri = fullPath |> getFileUri; spiText = code |} |}
    let! _fileOpenResult = fileOpenObj |> sendObj serverPort

    let fileTokenRangeObj =
        {|
            FileTokenRange =
                {|
                    uri = fullPath |> getFileUri
                    range =
                        [|
                            {| line = 0; character = 0 |}
                            {| line = lines.Length - 1; character = lines.[lines.Length - 1].Length |}
                        |]
                |}
        |}
    let! fileTokenRangeResult =
        fileTokenRangeObj
        |> sendObj serverPort
        |> Async.withCancellationToken ct

    return fileTokenRangeResult |> Option.map FSharp.Json.Json.deserialize<int array>
}

## getCodeTokenRange

In [ ]:
let inline getCodeTokenRange cancellationToken code = async {
    let! mainPath, disposable = persistCode code
    use _ = disposable
    let port = getCompilerPort ()
    return! mainPath |> getFileTokenRange port cancellationToken
}

In [ ]:
//// test

"""inl main () = ()"""
|> getCodeTokenRange None
|> Async.runWithTimeout 10000
|> Option.flatten
|> _equal (Some [| 0; 0; 3; 7; 0; 0; 4; 4; 0; 0; 0; 5; 1; 8; 0; 0; 1; 1; 8; 0; 0; 2; 1; 4; 0; 0;
2; 1; 8; 0; 0; 1; 1; 8; 0 |])

00:00:53 #461 [Debug] createTempDirectory / tempFolder: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20231008-2314-1774-7480-7d60cb466671 / result: { CreationTime = 2023-10-08 11:14:17 PM
  Exists = true }
00:00:54 #462 [Debug] runWithTimeoutAsync / timeout: 60
00:00:54 #463 [Debug] executeAsync / options: { Command =
   "dotnet "C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release\Spiral.dll" port=13805"
  WorkingDirectory = None
  CancellationToken = Some System.Threading.CancellationToken
  OnLine = Some <fun:it@4-374> }
00:00:54 #464 [Debug] > pwd: C:\home\git\polyglot\apps\spiral
00:00:54 #465 [Debug] > dll_path: C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release
00:00:54 #466 [Debug] runWithTimeoutAsync / timeout: 500
00:00:55 #467 [Debug] sendJson / port: 13805 / json: {"Ping":true} / result.Length:
00:00:55 #468 [Verbose] awaitCompiler / Ping / result: Some(

In [ ]:
//// test

"""inl main () = 1i32"""
|> getCodeTokenRange None
|> Async.runWithTimeout 10000
|> Option.flatten
|> _equal (Some [| 0; 0; 3; 7; 0; 0; 4; 4; 0; 0; 0; 5; 1; 8; 0; 0; 1; 1; 8; 0; 0; 2; 1; 4; 0; 0;
2; 1; 3; 0; 0; 1; 3; 12; 0 |])

00:01:00 #472 [Debug] createTempDirectory / tempFolder: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20231008-2314-2389-8926-86ced5859ce5 / result: { CreationTime = 2023-10-08 11:14:23 PM
  Exists = true }
00:01:00 #473 [Debug] runWithTimeoutAsync / timeout: 60
00:01:00 #474 [Debug] executeAsync / options: { Command =
   "dotnet "C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release\Spiral.dll" port=13805"
  WorkingDirectory = None
  CancellationToken = Some System.Threading.CancellationToken
  OnLine = Some <fun:it@4-741> }
00:01:00 #475 [Debug] > pwd: C:\home\git\polyglot\apps\spiral
00:01:00 #476 [Debug] > dll_path: C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release
00:01:00 #477 [Debug] runWithTimeoutAsync / timeout: 500
00:01:00 #478 [Verbose] waitForPortAccess / port: 13805 / retry: 0
00:01:01 #479 [Debug] sendJson / port: 13805 / json: {"Ping":true} / result.L

## Arguments

In [ ]:
[<RequireQualifiedAccess>]
type Arguments =
    | BuildFile of string * string
    | FileTokenRange of string * string
    | ExecuteCommand of string
    | Timeout of int
    | Port of int

    interface Argu.IArgParserTemplate with
        member s.Usage =
            match s with
            | BuildFile _ -> nameof BuildFile
            | FileTokenRange _ -> nameof FileTokenRange
            | ExecuteCommand _ -> nameof ExecuteCommand
            | Timeout _ -> nameof Timeout
            | Port _ -> nameof Port

## main

In [ ]:
let main args =
    let argsMap = args |> Runtime.parseArgsMap<Arguments>

    let buildFileActions =
        argsMap
        |> Map.tryFind (nameof Arguments.BuildFile)
        |> Option.defaultValue []
        |> List.choose (function
            | Arguments.BuildFile (inputPath, outputPath) -> Some (inputPath, outputPath)
            | _ -> None
        )

    let fileTokenRangeActions =
        argsMap
        |> Map.tryFind (nameof Arguments.FileTokenRange)
        |> Option.defaultValue []
        |> List.choose (function
            | Arguments.FileTokenRange (inputPath, outputPath) -> Some (inputPath, outputPath)
            | _ -> None
        )

    let executeCommandActions =
        argsMap
        |> Map.tryFind (nameof Arguments.ExecuteCommand)
        |> Option.defaultValue []
        |> List.choose (function
            | Arguments.ExecuteCommand command -> Some command
            | _ -> None
        )

    let timeout =
        match argsMap |> Map.tryFind (nameof Arguments.Timeout) with
        | Some [ Arguments.Timeout timeout ] -> timeout
        | _ -> 60000 * 5

    let port =
        match argsMap |> Map.tryFind (nameof Arguments.Port) with
        | Some [ Arguments.Port port ] -> Some port
        | _ -> None

    async {
        let! buildFileResult =
            buildFileActions
            |> List.map (fun (inputPath, outputPath) -> async {
                let port = port |> Option.defaultWith getCompilerPort
                let! outputCode, errors = inputPath |> buildFile timeout port None
                
                errors
                |> List.map snd
                |> List.iter (fun error ->
                    trace Critical (fun () -> $"main / error: {error}") getLocals
                )

                match outputCode with
                | Some outputCode ->
                    do! outputCode |> FileSystem.writeAllTextAsync outputPath
                    return 0
                | None ->
                    return 1
            })
            |> Async.Sequential

        let! fileTokenRangeResult =
            fileTokenRangeActions
            |> List.map (fun (inputPath, outputPath) -> async {
                let port = port |> Option.defaultWith getCompilerPort
                let! tokenRange = inputPath |> getFileTokenRange port None
                match tokenRange with
                | Some tokenRange ->
                    do! tokenRange |> FSharp.Json.Json.serialize |> FileSystem.writeAllTextAsync outputPath
                    return 0
                | None ->
                    return 1
            })
            |> Async.Sequential
        
        let! executeCommandResult =
            executeCommandActions
            |> List.map (fun command -> async {
                let port = port |> Option.defaultWith getCompilerPort

                let localToken, disposable = Threading.newDisposableToken None
                use _ = disposable

                let! serverPort, _errors, compilerToken, disposable = awaitCompiler port (Some localToken)
                use _ = disposable
                
                let! exitCode, result =
                    Runtime.executeWithOptionsAsync
                        {
                            Command = command
                            CancellationToken = Some compilerToken
                            WorkingDirectory = None
                            OnLine = None
                        }

                trace Debug (fun () -> $"main / executeCommand / exitCode: {exitCode}") getLocals

                return exitCode
            })
            |> Async.Sequential
            
        return
            [| buildFileResult; fileTokenRangeResult; executeCommandResult |]
            |> Array.collect id
            |> Array.sum
    }
    |> Async.runWithTimeout timeout
    |> Option.defaultValue 1

In [ ]:
//// test

let args =
    System.Environment.GetEnvironmentVariable "ARGS"
    |> Runtime.splitArgs
    |> Seq.toArray

match args with
| [||] -> 0
| args -> if main args = 0 then 0 else failwith "main failed"

0